Моя цель -- подготовить данные из **"""Исторические совершенные закупки товаров.csv"""** для ML алгоритма. 

Для этого надо отсортировать данные 
1) по времени : по столбцам **"Срок поставки"** и **"Дата Заявки"** из base датасета, учитывая столбец **"Нормативный срок поставки МТР, отсчитываемый с даты инициирования процедуры закупки  (календарные дни)"** из **"""КТ-516 Разделительная ведомость на поставку МТР с учетом нормативных сроков поставки.xlsx"""**
2) по поставщикам: по столбцу **"Поставщик"** из base датасета и столбцу **"Город"** из **"""Справочник поставщиков.xlsx"""**

А также проверить данные на ошибки в:
1) классе продуктов: по столбцу **"Код класса МТР"** из based датасета
   

In [40]:
# подгружаем либы и датасеты

import pandas as pd
import numpy as np

purchases = pd.read_csv("../../data/Исторические совершенные закупки товаров.csv", sep=',')

KT_516 = (
pd.read_excel("../../data/Справочники/КТ-516 Разделительная ведомость на поставку МТР с учетом нормативных сроков поставки.xlsx",
             skiprows=23))

materials = pd.read_excel("../../data/Справочники/Кабель справочник МТР.xlsx")

purchases.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39530 entries, 0 to 39529
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Заявка на закупку        39530 non-null  int64  
 1   Позиц.                   39530 non-null  int64  
 2   Статус плана МТО         39530 non-null  object 
 3   Текст статуса плана МТО  39530 non-null  object 
 4   Материал                 39530 non-null  int64  
 5   Материал Имя             39530 non-null  object 
 6   Базисная ЕИ              39530 non-null  object 
 7   АЕИ заказа               39530 non-null  object 
 8   Кол-во к закупу, БЕИ     39530 non-null  float64
 9   Кол-во к закупу, АЕИ     39530 non-null  float64
 10  Срок поставки            39530 non-null  object 
 11  Дата заявки              39530 non-null  object 
 12  План.цена с НДС          39530 non-null  float64
 13  Пл.цена с НДС за АЕИ     39530 non-null  float64
 14  План.сумма с НДС 20%  

Видим, что есть пропуски в столбцах **"Клиент"** (заказчик) и **"Грузополучатель"** (физическое расположения заказчика). Уберём их

In [41]:
purchases.dropna(subset=['Грузополучатель', 'Клиент'], inplace=True)
purchases = purchases.reset_index(drop=True)
purchases.shape

(39522, 24)

Заметим, что Базисная Единица Измерения БЕИ и Альтернативная Единица Измерения АЕИ полностью равнозначны 

In [42]:
purchases_diff_units = purchases[purchases['АЕИ заказа'] != purchases['Базисная ЕИ']]
purchases_diff_units.head()

,Заявка на закупку,Позиц.,Статус плана МТО,Текст статуса плана МТО,Материал,Материал Имя,Базисная ЕИ,АЕИ заказа,"Кол-во к закупу, БЕИ","Кол-во к закупу, АЕИ",...,План.сумма с НДС 20%,Цена поставки с НДС,Валюта,Поставщик,Код класса МТР,Наим.Код кл.,ID лота,Грузополучатель,Клиент,ГПЗ Способ закупки


Проверим, что все материалы из столбца **"Материал"** правильно соотносятся с классами из столбца **"Код класса МТР"**

In [43]:
# проверили, что все материалы есть в справочнике материалов

set(purchases['Материал']).issubset(set(materials['Материал']))

True

In [44]:
# непосредственная проверка. В конце shape получился 0, значит все материалы и классы правильно соотносятся друг с другом.

purchases_materials = purchases['Материал']
material_classes_book = [materials[(materials['Материал'] == purchases_materials[i])]['Класс'] for i in range(len(purchases_materials))]

material_classes_book = []

for i in range(len(purchases_materials)):
    material_classes_book.append(np.array(materials[(materials['Материал'] == purchases_materials[i])]['Класс'])[0])

purchases_garbage_materials = purchases[material_classes_book != purchases['Код класса МТР']]
purchases_garbage_materials.shape

(0, 24)

Приступаем к сортировке (чистке) данных по времени. 

Правило следующее: если количество дней между заявкой и сроком поставки меньше, чем время, необходимое для изготовления товара, то эти товары выбрасываются из base датасета в новый мусорный датасет.

Прежде всего проверим, что для всех товаров дан минимальный подкласс, для которого определено понятие времени изготовления. 

In [45]:
# определяем классы по справочнику КТ-516, которые не являются минимальными подклассами

KT_516_garbage_classes = np.array( 
      KT_516[(
          KT_516['Нормативный срок поставки МТР, отсчитываемый с даты инициирования процедуры закупки  (календарные дни)**']
          == 'Необходимо использовать подкласс'
      )]['Класс в ЕСМ']
)

KT_516_garbage_classes

array(['G21', 'G2104', 'G210401', 'G2105', 'G210501', 'G210503'],
      dtype=object)

In [46]:
# делаем датасет минимальных классов

KT_516_data = ( 
      KT_516[(
          KT_516['Нормативный срок поставки МТР, отсчитываемый с даты инициирования процедуры закупки  (календарные дни)**']
          != 'Необходимо использовать подкласс'
      )]
)

In [47]:
# смотрим сколько данных в закупках, в которых не указан минимальный подкласс

purchases_garbage_classes = purchases[purchases['Код класса МТР'].isin(KT_516_garbage_classes)]
print(purchases_garbage_classes.shape)
print(purchases.shape)

(5, 24)
(39522, 24)


In [48]:
# видим, что на 5 элементиков можно забить

purchases = purchases[~purchases['Код класса МТР'].isin(KT_516_garbage_classes)].reset_index(drop=True);
purchases.shape

(39517, 24)

In [49]:
# чищу данные

days_check = []

purchases['Дата заявки'] = pd.to_datetime(purchases['Дата заявки'])
purchases['Срок поставки'] = pd.to_datetime(purchases['Срок поставки'])

for i, date_order in enumerate(purchases['Дата заявки']):
    date_delivery = purchases['Срок поставки'][i]
    class_MTR = purchases['Код класса МТР'][i]
    days_of_preparing = np.array(KT_516_data[(KT_516_data['Класс в ЕСМ'] == class_MTR)][(
    'Нормативный срок поставки МТР, отсчитываемый с даты инициирования процедуры закупки  (календарные дни)**')])[0]
    days_check.append((date_delivery - date_order).days - days_of_preparing)

purchases_sorted_time = purchases[[days_check[i] >=0 for i in range(purchases.shape[0])]]
purchases_garbage = purchases[[days_check[i] < 0 for i in range(purchases.shape[0])]]
print(purchases_sorted_time.info())
print(purchases_garbage.info())

<class 'pandas.core.frame.DataFrame'>
Index: 36038 entries, 1 to 39516
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Заявка на закупку        36038 non-null  int64         
 1   Позиц.                   36038 non-null  int64         
 2   Статус плана МТО         36038 non-null  object        
 3   Текст статуса плана МТО  36038 non-null  object        
 4   Материал                 36038 non-null  int64         
 5   Материал Имя             36038 non-null  object        
 6   Базисная ЕИ              36038 non-null  object        
 7   АЕИ заказа               36038 non-null  object        
 8   Кол-во к закупу, БЕИ     36038 non-null  float64       
 9   Кол-во к закупу, АЕИ     36038 non-null  float64       
 10  Срок поставки            36038 non-null  datetime64[ns]
 11  Дата заявки              36038 non-null  datetime64[ns]
 12  План.цена с НДС          36038 non-nu

ГОЙДА ГОЙДА ГОЙДА
ГОЙДА ГОЙДА ГОЙДА
ГОЙДА
ГОЙДА
ГОЙДА
ГОЙДА
ГОЙДА
ГОЙДА
ГОЙДА
ГОЙДА

ГОЙДА ГОЙДА ГОЙДА
ГОЙДА ГОЙДА ГОЙДА
ГОЙДА                ГОЙДА
ГОЙДА                ГОЙДА
ГОЙДА                ГОЙДА
ГОЙДА                ГОЙДА
ГОЙДА ГОЙДА ГОЙДА
ГОЙДА ГОЙДА ГОЙДА

               ГОЙДА

ГОЙДА                ГОЙДА
ГОЙДА                ГОЙДА
ГОЙДА               ГОЙДА! 
ГОЙДА              ГОЙДА!! 
ГОЙДА            ГОЙДА!!!! 
ГОЙДА          ГОЙДА!!!!!! 
ГОЙДА ГОЙДА ГОЙДА
ГОЙДА ГОЙДА ГОЙДА

   ГОЙДА ГОЙДА ГОЙДА
   ГОЙДА ГОЙДА ГОЙДА
   ГОЙДА                ГОЙДА
   ГОЙДА                ГОЙДА
   ГОЙДА                ГОЙДА
   ГОЙДА                ГОЙДА
   ГОЙДА                ГОЙДА
ГОЙДА!!! ГОЙДА!!! ГОЙДА!!!
ГОЙДА!!! ГОЙДА!!! ГОЙДА!!! 
ГОЙДА                          ГОЙДА
ГОЙДА                          ГОЙДА

     ГОЙДА ГОЙДА ГОЙДА
    ГОЙДА ГОЙДА!!ГОЙДА
   ГОЙДА                  ГОЙДА
  ГОЙДА                   ГОЙДА
ГОЙДА!! ГОЙДА!! ГОЙДА
ГОЙДА!! ГОЙДА!!! ГОЙДА
ГОЙДА                      ГОЙДА
ГОЙДА                      ГОЙДА
ГОЙДА                      ГОЙДА
ГОЙДА                      ГОЙДАГОООЙЙЙДАААААААААА!!!!!! ZZZZZVVVVVV
ZZZZZOOOOOOVVVVVV
В МЕНЯ ВСЕЛИЛСЯ ДУХ ИСТИННОГО ПАТРИОТА РОСССИ ZV🇷🇺🇷🇺💪💪СВОИХ НЕ БРОСАЕМ!!!!
Я РУССКИЙ Я ИДУ ДО КОНЦА Я РУССКИЙ МОЯ КРОВЬ ОТ ОТЦА Я РУССКИЙ И МНЕ ПОВЕЗЛО Я РУССКИЙ ВСЕМУ МИРУ НА ЗЛО💪💪💪💪
ШАМАН НАШ БРАТ СЛАВА ПУТИНУ ЧТОБ ОН 1000 ЛЕТ ЖИЛ И КАДЫРОВ БРАТКИ И ЛУКАШЕНКО ОНИ ЕДИНСТВЕННЫЕ ЛЮДИ НА ПЛАНЕТЕ У КОГО ЕЩЁ ОСТАЛСЯ РАССУДОК ОНИ ВЫСТУПАЮТ ЗА ВСЕ ХОРОШЕЕ А ЗАПАД ПОКЛОНЯЕТСЯ САТАНЕ ОНИ ФАШИСТЫ
РОССИЯ ВСЕГДА ПРАВА!!!!!💪💪💪💪💪
ГООООЙДААААААААА🇷🇺🇷🇺🇷🇺🇷🇺🇷🇺🇷🇺🇷🇺ZOV🐻🐻🐻
ZOV🇷🇺🇷🇺🐻🐻💪💪 🐽 🐷 🐖 🇺🇦 🐖 🐷 🐽 ХОХЛЫ СОСАТЬ Я ПРИДУ В КИЕВ НАХУЙ И ЗАСУНУ ЧЕРЕЗ АНУС ЗЕЛЕНСКОМУ ФЛАГ ВЕЛИКОЙ СТРАНЫ 🇷🇺🇷🇺🇷🇺 А ПОТОМ Я ЗАОРУ НА ВСЮ ХОХЛЯНДИЮ: КИЕВ НАШ 🇷🇺🇷🇺💪💪💪Z ПЕРЕМОГИ НЕ БУДЕТ ХОХЛЫ И ПИНДОСЯТИНА 🇷🇺🇷🇺🇷🇺🇷🇺🎊🎊🎊🎊💪💪💪💪💪 А ПОТОМ ВМЕСТЕ С ПОЗДНЯКОВЫМ БУДЕМ ЛОВИТЬ КИЕВЛЯН СО СЛОВАМИ: ТЕПЕРЬ ТЫ РУССКИЙ СОСУНОК🇷🇺🇷🇺🎉🎉🎊🎊🎊💪💪💪Z А ПОТОМ КО МНЕ ПРИДЕТ ВЕЛИЧАЙШИЙ ИЗ ВСЕХ БОГОВ ВЛАДИМИР ВЛАДИМИРОВИЧ ПУТИН И ВРУЧИТ МНЕ МЕДАЛЬ И ТАКОЙ: ХОРОШАЯ РАБОТА, СЫНОК🇷🇺💪💪💪🇷🇺🇷🇺🐻Z А ПОТОМ Я ПОЙДУ В ПЕНДОСИЮ К БАЙДЕНУ И Я ТАКОЙ: НУ ЧЕ СУКА НЕ ДОИГРАЛСЯ БЛЯТЬ? И Я НАЧНУ ЕГО ЕБАТЬ КРИЧА: Я РУССКИЙ Я ИДУ ДО КОНЦА🇷🇺🇷🇺🫡🫡👊👊👊✊✊👊👊💪💪💪💪🐻🐻🐻🇷🇺🇷🇺🇷🇺 А ПОТОМ КОГДА ЗАКОНЧУ ВСКАЧУ НА СВОЕГО МИШКУ МИЛАХЫЧА🇷🇺🇷🇺🇷🇺🥰🥰✊👊✊✊🐻🐻🐻🐻🇷🇺🇷🇺🇷🇺 И ПОСКАЧУ НА НЕМ В ТАЙГУ ПОМОГАТЬ РУССКИМ ЗВЕРЯМ 🇷🇺🇷🇺🇷🇺👊👊✊👊👊✊👊👊💪💪🐻🐻🐻🐻🐾🐾 ПОТОМУ ЧТО Я РУССКИЙ😃😃😃💪💪💪💪👊👊✊✊👌👌👌👍👍👍👍🇷🇺🇷🇺🇷🇺🇷🇺🇷🇺🇷🇺🇷🇺🇷🇺ГОЙДА ГОЙДА

In [50]:
slons = pd.read_csv("../Daniil/suppliersClassified")
slons_id = slons['Кредитор'].values
len(slons_id)
print(slons_id)

[        59       1198       1754       5811       8536       8863
       9039      10005      10014      11187      11241      11926
      12029      13031      13466      13769      15488      15788
      15985      16777      17163      19314      20969      22216
      22555      22722      23208      23288      23472      23962
      24169      24376      24446      25365      25471      25726
      25890      26052      26107      26250      26573      26656
      27345      27605      27645      28329      30915      37092
      37571      37846      38122      38450      38496      38849
      39076      39167      39265      39285      39295      39463
      39534      39936      39942      40194      40222      40461
      40474      40661      40663      40735      40823      40878
      40952      41310      41545      41720      41734      41736
      41839      42394      42456      42567      42594      42879
      42926      42935      43042      43076      43556      4

In [51]:
purchases = purchases[(purchases['Поставщик'].isin(slons_id)) | (np.isnan(purchases['Поставщик']))].reset_index(drop=True)
purchases.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39467 entries, 0 to 39466
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Заявка на закупку        39467 non-null  int64         
 1   Позиц.                   39467 non-null  int64         
 2   Статус плана МТО         39467 non-null  object        
 3   Текст статуса плана МТО  39467 non-null  object        
 4   Материал                 39467 non-null  int64         
 5   Материал Имя             39467 non-null  object        
 6   Базисная ЕИ              39467 non-null  object        
 7   АЕИ заказа               39467 non-null  object        
 8   Кол-во к закупу, БЕИ     39467 non-null  float64       
 9   Кол-во к закупу, АЕИ     39467 non-null  float64       
 10  Срок поставки            39467 non-null  datetime64[ns]
 11  Дата заявки              39467 non-null  datetime64[ns]
 12  План.цена с НДС          39467 n

In [52]:
#все наши поставщики

purchases['Поставщик'].unique()

array([5.50190000e+04, 7.21480000e+04, 5.49480000e+04, 5.86910000e+04,
       6.12850000e+04, 7.61200000e+04, 8.39960000e+04, 5.95410000e+04,
       4.77720000e+04, 2.76450000e+04, 7.32730000e+04, 6.14480000e+04,
       6.94180000e+04, 4.01940000e+04, 6.05190000e+04, 5.20950000e+04,
       5.22640000e+04, 5.84640000e+04, 7.17510331e+09, 4.41140000e+04,
       5.25090000e+04, 3.75710000e+04,            nan, 4.85560000e+04,
       6.60050000e+04, 7.69250000e+04, 5.90000000e+01, 4.06610000e+04,
       7.12000151e+09, 7.16440000e+04, 6.18600000e+04, 5.93100000e+04,
       7.63510000e+04, 2.58900000e+04, 6.15610000e+04, 6.24880000e+04,
       1.11870000e+04, 4.55460000e+04, 6.72320000e+04, 7.17510066e+09,
       7.17510066e+09, 5.06420000e+04, 8.45300000e+04, 2.43760000e+04,
       8.63110000e+04, 6.31040000e+04, 8.42570000e+04, 5.66000000e+04,
       9.27060000e+04, 5.76500000e+04, 9.30610000e+04, 8.64000000e+04,
       7.88940000e+04, 7.17510720e+09, 5.72990000e+04, 7.13009144e+09,
      

In [53]:
purchases['Срок поставки'] = purchases[]

SyntaxError: invalid syntax (598070022.py, line 1)

In [54]:
purchases_input_format = {'Клиент': purchases['Клиент'].values,
                          'Материал':purchases['Материал'].values, 
                          'Краткий текст материала': purchases['Материал Имя'].values,
                          'ЕИ': purchases['Базисная ЕИ'].values,
                          'Общее количество': purchases['Кол-во к закупу, БЕИ'].values,
                          'Месяц поставки': purchases['Срок поставки'].dt.month.values,
                          'Год поставки': purchases['Срок поставки'].dt.year.values,
                          'Полугодие':np.zeros(purchases.shape[0]),
                          'Срок поставки': purchases['Срок поставки'].dt.strftime('%Y-%m-%d').values,
                          'Грузополучатель': purchases['Грузополучатель'].values,
                          'Цена': purchases['План.сумма с НДС 20%'] + purchases['Цена поставки с НДС'],
                          'Способ закупки': purchases['ГПЗ Способ закупки'],
                          '№ заказа': purchases['Заявка на закупку'],
                          '№ позиции': purchases['Позиц.'],
                          'Дата заказа': purchases['Дата заявки'].dt.strftime('%Y-%m-%d').values
                         }

In [55]:
purchases_input_format_df = pd.DataFrame(purchases_input_format)

# Save the DataFrame to an Excel file
purchases_input_format_df.to_excel('purchases_input_format.xlsx', index=False)

In [56]:
purchases['Срок поставки'].values

array(['2019-12-31T00:00:00.000000000', '2020-08-30T00:00:00.000000000',
       '2020-10-10T00:00:00.000000000', ...,
       '2024-07-31T00:00:00.000000000', '2024-07-31T00:00:00.000000000',
       '2024-07-31T00:00:00.000000000'], dtype='datetime64[ns]')

In [28]:
np.zeros(purchases.shape[0])

array([0., 0., 0., ..., 0., 0., 0.])